In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import joblib


In [2]:
# joblib.dump(data, 'data_with_features.joblib')
data = joblib.load('data.joblib')
data

,ECNumber,Organism,Smiles,Substrate,Sequence,Raw Kd,Unit,Log Kd,metabolite_features,protein_features,label
0,4.2.3.4,Actinidia chinensis,C(C(C(C(COP(=O)(O)O)O)O)O)C(=O)C(=O)O,3-deoxy-D-arabino-heptulosonic acid 7-phosphate,MAAFSLSAKQILSPSTHRPSLSKTTTADSSLRFRNPHSLSLRCSSL...,20.1000,s^(-1),1.303196,"[0.09045384, -0.6035856, -0.5294743, 0.0703877...","[-0.12646541, -0.15512069, 0.0834027, 0.213126...",-1.303196
1,4.2.3.4,Aspergillus nidulans,C(C(C(C(COP(=O)([O-])[O-])O)O)O)C(=O)C(=O)[O-],3-deoxy-D-arabino-heptulosonate 7-phosphate,MSNPTKISILGRESIIADFGLWRNYVAKDLISDCSSTTYVLVTDTN...,6.8000,s^(-1),0.832509,"[0.4610253, 0.039623156, -0.4662559, 0.4188257...","[-0.10084063, -0.099633686, -0.011382125, 0.10...",-0.832509
2,4.2.3.4,Neurospora crassa,C(C(C(C(COP(=O)(O)O)O)O)O)C(=O)C(=O)O,3-deoxy-D-arabino-heptulosonic acid 7-phosphate,MAEPISNPTRINILGKDNIIIDHGIWLNFVAQDLLQNIKSSTYILI...,19.0000,s^(-1),1.278754,"[0.09045384, -0.6035856, -0.5294743, 0.0703877...","[-0.12705962, -0.09898393, 0.0041820942, 0.096...",-1.278754
3,2.1.1.255,Streptomyces coelicolor,C[S+](CCC(C(=O)[O-])N)CC1C(C(C(O1)N2C=NC3=C(N=...,S-Adenosyl-L-methionine,MTTETTTATATAKIPAPATPYQEDIARYWNNEARPVNLRLGDVDGL...,0.0075,s^(-1),-2.124939,"[0.52348095, -0.18665986, -0.39555183, 0.11716...","[0.0015065962, -0.087554656, -0.014490904, -0....",2.124939
4,2.1.1.255,Streptomyces coelicolor,CC(=CCCC(=C(C)COP(=O)(O)OP(=O)(O)O)C)C,(E)-2-Methylgeranyl diphosphate,MTTETTTATATAKIPAPATPYQEDIARYWNNEARPVNLRLGDVDGL...,0.0390,s^(-1),-1.408935,"[0.17630623, -0.4595529, -0.6368343, -0.136045...","[0.0015065962, -0.087554656, -0.014490904, -0....",1.408935
...,...,...,...,...,...,...,...,...,...,...,...
17005,1.1.1.82,Zea mays,C1C=CN(C=C1C(=O)N)C2C(C(C(O2)COP(=O)(O)OP(=O)(...,NADPH,MGLSTVYSPAGPRLVPAPLGRCRSAQPRRPRRAPLATVRCSVDATK...,955.0000,s^(-1),2.980003,"[0.36284313, -0.21340896, -0.8913718, -0.23485...","[0.039386757, -0.04558141, 0.101506, 0.1012840...",-2.980003
17006,1.1.1.82,Zea mays,C(C(C(=O)O)O)C(=O)O,L-Malate,MGLSTVYSPAGPRLVPAPLGRCRSAQPRRPRRAPLATVRCSVDATK...,2.9000,s^(-1),0.462398,"[0.045168612, -0.31311044, -0.21087039, -0.292...","[0.039386757, -0.04558141, 0.101506, 0.1012840...",-0.462398
17007,1.1.1.82,Zea mays,C1=CC(=C[N+](=C1)C2C(C(C(O2)COP(=O)(O)OP(=O)(O...,NADP+,MGLSTVYSPAGPRLVPAPLGRCRSAQPRRPRRAPLATVRCSVDATK...,2.9000,s^(-1),0.462398,"[0.18913552, -0.06542938, -0.8761325, 0.133454...","[0.039386757, -0.04558141, 0.101506, 0.1012840...",-0.462398
17008,1.1.1.82,Spinacia oleracea,C(C(C(=O)O)O)C(=O)O,L-Malate,MAVAELSPCYQTQIVKPPHLSWLSNNHKLNLLGLPKASRITEICCS...,6.7000,s^(-1),0.826075,"[0.045168612, -0.31311044, -0.21087039, -0.292...","[0.029217303, -0.08372673, 0.0883128, 0.147737...",-0.826075


In [3]:
# adding 1e-9 to prevent error when doing logarithm
data['label'] = -np.log10(data['Raw Kd'] + 1e-9)

In [4]:
# Define dataset
class MPI_Dataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        return {
            'metabolite_features': torch.tensor(np.asarray(row['metabolite_features'], dtype=np.float32)),
            'protein_features': torch.tensor(np.asarray(row['protein_features'], dtype=np.float32)),
            'label': torch.tensor(float(row['label']), dtype=torch.float32),
        }

In [5]:
data.columns

Index(['ECNumber', 'Organism', 'Smiles', 'Substrate', 'Sequence', 'Raw Kd',
       'Unit', 'Log Kd', 'metabolite_features', 'protein_features', 'label'],
      dtype='object')

In [6]:
# Separate the dataset by unique proteins and drugs
unique_proteins = data['Sequence'].unique()
unique_mols = data['Smiles'].unique()
# print(unique_proteins)
# print(unique_mols)
# Set the seed for reproducibilityity
torch.manual_seed(42)
# Function to perform a cold split
def cold_split(unique_items, test_size=0.2, val_size=0.1):
    train_items, test_items = train_test_split(unique_items, test_size=test_size, random_state=42)
    train_items, val_items = train_test_split(train_items, test_size=val_size / (1 - test_size), random_state=42)
    return train_items, val_items, test_items
# Cold split by proteins
train_proteins, val_proteins, test_proteins = cold_split(unique_proteins)
train_cold_protein = data[data['Sequence'].isin(train_proteins)]
val_cold_protein = data[data['Sequence'].isin(val_proteins)]
test_cold_protein = data[data['Sequence'].isin(test_proteins)]
# Cold split by molecules
train_mols, val_mols, test_mols = cold_split(unique_mols)
train_cold_mols = data[data['Smiles'].isin(train_mols)]
val_cold_mols = data[data['Smiles'].isin(val_mols)]
test_cold_mols = data[data['Smiles'].isin(test_mols)]

In [7]:
def df2array(df):
    X = np.array([
    np.concatenate([m, p])
    for m, p in zip(df['metabolite_features'], df['protein_features'])])
    y = df['label']
    return X, y

train_X, train_y = df2array(train_cold_protein)
val_X, val_y = df2array(val_cold_protein)
test_X, test_y = df2array(test_cold_protein)

train_X_mols, train_y_mols = df2array(train_cold_mols)
val_X_mols, val_y_mols = df2array(val_cold_mols)
test_X_mols, test_y_mols = df2array(test_cold_mols)


In [8]:
import json
import numpy as np

# 創建 cold protein 索引資訊
cold_protein_index_info = {
    "train_index": train_cold_protein.index.tolist(),
    "val_index": val_cold_protein.index.tolist(),
    "test_index": test_cold_protein.index.tolist(),
    "train_proteins": train_proteins.tolist(),
    "val_proteins": val_proteins.tolist(),
    "test_proteins": test_proteins.tolist(),
    "total_train_samples": len(train_cold_protein),
    "total_val_samples": len(val_cold_protein),
    "total_test_samples": len(test_cold_protein),
    "split_info": {
        "split_type": "cold_protein",
        "test_size": 0.2,
        "val_size": 0.1,
        "random_state": 42
    }
}

# 創建 cold mols 索引資訊
cold_mols_index_info = {
    "train_index": train_cold_mols.index.tolist(),
    "val_index": val_cold_mols.index.tolist(),
    "test_index": test_cold_mols.index.tolist(),
    "train_mols": train_mols.tolist(),
    "val_mols": val_mols.tolist(),
    "test_mols": test_mols.tolist(),
    "total_train_samples": len(train_cold_mols),
    "total_val_samples": len(val_cold_mols),
    "total_test_samples": len(test_cold_mols),
    "split_info": {
        "split_type": "cold_mols",
        "test_size": 0.2,
        "val_size": 0.1,
        "random_state": 42
    }
}

# 保存到 JSON 檔案
with open('cold_protein_index_info.json', 'w', encoding='utf-8') as f:
    json.dump(cold_protein_index_info, f, indent=2, ensure_ascii=False)

with open('cold_mols_index_info.json', 'w', encoding='utf-8') as f:
    json.dump(cold_mols_index_info, f, indent=2, ensure_ascii=False)

print("索引資訊已保存到:")
print("- cold_protein_index_info.json")
print("- cold_mols_index_info.json")

# 驗證保存的資料
print(f"\nCold Protein Split:")
print(f"  Train: {len(train_cold_protein)} samples, {len(train_proteins)} unique proteins")
print(f"  Val: {len(val_cold_protein)} samples, {len(val_proteins)} unique proteins")
print(f"  Test: {len(test_cold_protein)} samples, {len(test_proteins)} unique proteins")

print(f"\nCold Mols Split:")
print(f"  Train: {len(train_cold_mols)} samples, {len(train_mols)} unique molecules")
print(f"  Val: {len(val_cold_mols)} samples, {len(val_mols)} unique molecules")
print(f"  Test: {len(test_cold_mols)} samples, {len(test_mols)} unique molecules")

索引資訊已保存到:
- cold_protein_index_info.json
- cold_mols_index_info.json

Cold Protein Split:
  Train: 12084 samples, 5499 unique proteins
  Val: 1592 samples, 786 unique proteins
  Test: 3334 samples, 1572 unique proteins

Cold Mols Split:
  Train: 11813 samples, 1893 unique molecules
  Val: 1689 samples, 271 unique molecules
  Test: 3508 samples, 542 unique molecules


In [8]:
#model initialization
import catboost as cat
ca = cat.CatBoostRegressor()
ca_2 = cat.CatBoostRegressor()

In [9]:
'''
#Grid search
#ref: https://xgboost.readthedocs.io/en/stable/parameter.html
from sklearn.model_selection import GridSearchCV
param_grid = dict(
    max_depth = [5,10,15,20,None],
    n_estimators = [50,100,150,200]
)
regressor = GridSearchCV(estimator = xgb, param_grid = param_grid, cv = pds, verbose = 3, n_jobs = -1)
'''
#Train the model
ca.fit(train_X,train_y)
ca_2.fit(train_X_mols,train_y_mols)

Learning rate set to 0.060697
0:	learn: 1.4945964	total: 194ms	remaining: 3m 14s
1:	learn: 1.4841764	total: 234ms	remaining: 1m 56s
2:	learn: 1.4743716	total: 269ms	remaining: 1m 29s
3:	learn: 1.4649777	total: 304ms	remaining: 1m 15s
4:	learn: 1.4559407	total: 341ms	remaining: 1m 7s
5:	learn: 1.4471245	total: 375ms	remaining: 1m 2s
6:	learn: 1.4384602	total: 411ms	remaining: 58.3s
7:	learn: 1.4302692	total: 446ms	remaining: 55.3s
8:	learn: 1.4233142	total: 482ms	remaining: 53s
9:	learn: 1.4167050	total: 518ms	remaining: 51.3s
10:	learn: 1.4104126	total: 555ms	remaining: 49.9s
11:	learn: 1.4039804	total: 592ms	remaining: 48.8s
12:	learn: 1.3979053	total: 630ms	remaining: 47.8s
13:	learn: 1.3918442	total: 668ms	remaining: 47s
14:	learn: 1.3846252	total: 705ms	remaining: 46.3s
15:	learn: 1.3790219	total: 743ms	remaining: 45.7s
16:	learn: 1.3721449	total: 780ms	remaining: 45.1s
17:	learn: 1.3662656	total: 815ms	remaining: 44.5s
18:	learn: 1.3616814	total: 866ms	remaining: 44.7s
19:	learn: 

In [17]:
data_cold_protein = data.copy()
data_cold_mols = data.copy()
# data_cold_protein['pred'] = ca.predict(df2array(data_cold_protein)[0])
# data_cold_mols['pred'] = ca_2.predict(df2array(data_cold_mols)[0])
import os
prefix="CAT"
if os.path.exists('data_cold_protein.joblib'):
    data_cold_protein = joblib.load('data_cold_protein.joblib')
    if prefix+"_pred" in data_cold_protein.columns:
        raise ValueError("data_cold_protein already has a prediction column.")
    else:
        data_cold_protein[prefix+"_pred"] = ca.predict(df2array(data_cold_protein)[0])
else:
    data_cold_protein[prefix+"_pred"] = ca.predict(df2array(data_cold_protein)[0])
joblib.dump(data_cold_protein, 'data_cold_protein.joblib')

if os.path.exists('data_cold_mols.joblib'):
    data_cold_mols = joblib.load('data_cold_mols.joblib')
    if prefix+"_pred" in data_cold_mols.columns:
        raise ValueError("data_cold_mols already has a prediction column.")
    else:
        data_cold_mols[prefix+"_pred"] = ca_2.predict(df2array(data_cold_mols)[0])
else:
    data_cold_mols[prefix+"_pred"] = ca_2.predict(df2array(data_cold_mols)[0])
joblib.dump(data_cold_mols, 'data_cold_mols.joblib')


['data_cold_mols.joblib']

In [ ]:




# Train
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error, explained_variance_score
from scipy.stats import pearsonr

def metrics_cal(model,X,y):
    y_pred = model.predict(X)
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    pearson_corr, _ = pearsonr(y, y_pred)
    median_ae = median_absolute_error(y, y_pred)
    explained_var = explained_variance_score(y, y_pred)
    return mse, rmse, mae, r2, pearson_corr, median_ae, explained_var


train_mse, train_rmse, train_mae, train_r2, train_pearson_corr, train_median_ae, train_explained_var = metrics_cal(ca, train_X, train_y)
val_mse, val_rmse, val_mae, val_r2, val_pearson_corr, val_median_ae, val_explained_var = metrics_cal(ca, val_X, val_y)
test_mse, test_rmse, test_mae, test_r2, test_pearson_corr, test_median_ae, test_explained_var = metrics_cal(ca, test_X, test_y)


In [18]:
train_mse_mols, train_rmse_mols, train_mae_mols, train_r2_mols, train_pearson_corr_mols, train_median_ae_mols, train_explained_var_mols = metrics_cal(ca_2, train_X_mols, train_y_mols)
val_mse_mols, val_rmse_mols, val_mae_mols, val_r2_mols, val_pearson_corr_mols, val_median_ae_mols, val_explained_var_mols = metrics_cal(ca_2, val_X_mols, val_y_mols)
test_mse_mols, test_rmse_mols, test_mae_mols, test_r2_mols, test_pearson_corr_mols, test_median_ae_mols, test_explained_var_mols = metrics_cal(ca_2, test_X_mols, test_y_mols)

In [19]:
# append the performance to the csv file
df = {
    'Model':['CatBoost','CatBoost','CatBoost'],
    'Dataset':['Train','Validation','Test'],
    'MSE':[train_mse,val_mse,test_mse],
    'RMSE':[train_rmse,val_rmse,test_rmse],
    'MAE':[train_mae,val_mae,test_mae],
    'R2':[train_r2,val_r2,test_r2],
    'Pearson':[train_pearson_corr,val_pearson_corr,test_pearson_corr],
    'Median_AE':[train_median_ae,val_median_ae,test_median_ae],
    'Explained_VAR':[train_explained_var,val_explained_var,test_explained_var],
    'Dataspliting Mode':['cold protein','cold protein','cold protein']
}
df = pd.DataFrame(df)

df.to_csv('/Users/pinchichen/2025S lab/AI drug project/code/model performance metrics.csv', mode='a', header=False)

In [20]:
# append the performance to the csv file
df = {
    'Model':['CatBoost','CatBoost','CatBoost'],
    'Dataset':['Train','Validation','Test'],
    'MSE':[train_mse_mols,val_mse_mols,test_mse_mols],
    'RMSE':[train_rmse_mols,val_rmse_mols,test_rmse_mols],
    'MAE':[train_mae_mols,val_mae_mols,test_mae_mols],
    'R2':[train_r2_mols,val_r2_mols,test_r2_mols],
    'Pearson':[train_pearson_corr_mols,val_pearson_corr_mols,test_pearson_corr_mols],
    'Median_AE':[train_median_ae_mols,val_median_ae_mols,test_median_ae_mols],
    'Explained_VAR':[train_explained_var_mols,val_explained_var_mols,test_explained_var_mols],
    'Dataspliting Mode':['cold mols','cold mols','cold mols']
}
df = pd.DataFrame(df)

df.to_csv('/Users/pinchichen/2025S lab/AI drug project/code/model performance metrics.csv', mode='a', header=False)

In [22]:
# save the model
ca.save_model("/Users/pinchichen/2025S lab/AI drug project/code/trained_model/cold_protein/CatBoost model_cold_protein.json", format="json")
ca_2.save_model("/Users/pinchichen/2025S lab/AI drug project/code/trained_model/cold_mols/CatBoost model_cold_mols.json",format='json')

# load the model
#model = CatBoostRegressor()
#model.load_model("catboost_model.cbm"